In [34]:
#Feel free to modify this code as you see fit.

#NOTE: Some stations that are marked as separate are actually part of the same "complex." e.g. 59 ST & 59 ST COLUMBUS
#I haven't found a way to automatically combine their entries in the dataframe.

import requests
import pandas as pd

#You can change the link if you want the total traffic at an earlier date.
#To get the traffic from April 3 to July 10, you'd subtract the outputted numbers of April 3 from July 10.
#You'd need to mess around with the source code quite a bit; not that that's a bad thing.
r=requests.get('http://web.mta.info/developers/data/nyct/turnstile/turnstile_210710.txt')

#Contains every line in the archive. 
lines=r.text.split('\n')



latest=[]

for line in lines:
    #Set this line to the date/timestamp you want to pull from the data. MM/DD/YYYY,HH:00:00
    #Timestamps are taken every four hours.
    if line.split("07/09/2021,20:00:00")[0]!=line:
        l=line.split(',')
        #Stores relevant data in a tuple, and appends it to the list.
        #This line keeps the station name, entries, exits, turnstile ID aka SCP, and date/timestamp.
        latest.append((l[3].strip(),l[9].strip(),l[10].strip(),l[2].strip(),l[6].strip(),l[7].strip()))


In [35]:
data_dict={}
unique_stations=[]

for datapoint in latest:
    if datapoint[0] in unique_stations:
        pass
    else:
        #Create an entry in the dict for every unique station.
        unique_stations.append(datapoint[0])
        data_dict[datapoint[0]]=[]

#Fill the dict with data for entries and exits.
for datapoint in latest:
    data_dict[datapoint[0]].append((int(datapoint[1]),int(datapoint[2]), datapoint[3], datapoint[4], datapoint[5]))

In [36]:
#Tally up the lifetime entries and exits for every station.

tally_dict={}

for station in data_dict:
    tally_dict[station]=[]
    entries=0
    exits=0
    for turnstile in data_dict[station]:
        #print(station, turnstile)
        entries+=turnstile[0]
        exits+=turnstile[1]
    #Add Exits minus Entries, and Exit-Entry ratio to the dict, just in case anyone finds it helpful.
    tally_dict[station]=(entries,exits,exits-entries, exits/entries)

#Create a Pandas dataframe from the dictionary, and make the column names more helpful
mta_df=pd.DataFrame.from_dict(tally_dict,orient='index')
mta_df.rename(columns = {0:"ENTRIES"}, inplace = True)
mta_df.rename(columns = {1:"EXITS"}, inplace = True)
mta_df.rename(columns = {2:"EXITS LESS ENTRIES"}, inplace = True)
mta_df.rename(columns = {3:"EXIT-ENTRY RATIO"}, inplace = True)

In [47]:
for i in range(0,210):
    print(mta_df.iloc[i].name)

59 ST
57 ST-7 AV
49 ST
TIMES SQ-42 ST
34 ST-HERALD SQ
28 ST
23 ST
8 ST-NYU
PRINCE ST
CITY HALL
RECTOR ST
BOWERY
CHAMBERS ST
PROSPECT PARK
CHURCH AV
BEVERLEY ROAD
CORTELYOU RD
AVENUE H
AVENUE J
AVENUE M
KINGS HWY
NECK RD
BRIGHTON BEACH
OCEAN PKWY
BOROUGH HALL
JAY ST-METROTEC
DEKALB AV
ATL AV-BARCLAY
UNION ST
4AV-9 ST
PROSPECT AV
25 ST
45 ST
53 ST
BAY RIDGE AV
77 ST
86 ST
BAY RIDGE-95 ST
50 ST
55 ST
71 ST
79 ST
18 AV
20 AV
W 8 ST-AQUARIUM
6 AV
3 AV
1 AV
BEDFORD AV
GRAHAM AV
GRAND ST
MONTROSE AV
MORGAN AV
JEFFERSON ST
MYRTLE-WYCKOFF
HALSEY ST
WILSON AV
BUSHWICK AV
ATLANTIC AV
75 ST-ELDERTS
85 ST-FOREST PK
WOODHAVEN BLVD
104 ST
111 ST
121 ST
METROPOLITAN AV
190 ST
175 ST
145 ST
125 ST
96 ST
72 ST
59 ST COLUMBUS
42 ST-PORT AUTH
34 ST-PENN STA
SPRING ST
CANAL ST
WORLD TRADE CTR
HIGH ST
HOYT-SCHER
CLINTON-WASH AV
NOSTRAND AV
KINGSTON-THROOP
BROADWAY JCT
EUCLID AV
GRANT AV
OZONE PK LEFFRT
AQUEDUCT RACETR
HOWARD BCH JFK
BEACH 67 ST
BEACH 60 ST
BEACH 44 ST
BEACH 36 ST
BEACH 25 ST
FAR ROCKAWAY
16

In [43]:
mta_df.iloc[0:210]

,ENTRIES,EXITS,EXITS LESS ENTRIES,EXIT-ENTRY RATIO
59 ST,1395025097,925498419,-469526678,0.663428
57 ST-7 AV,961283836,523523131,-437760705,0.544608
49 ST,92777255,144039162,51261907,1.552527
TIMES SQ-42 ST,3641625441,2973107703,-668517738,0.816423
34 ST-HERALD SQ,3173693817,3894988464,721294647,1.227273
...,...,...,...,...
PRESIDENT ST,10356497,14181042,3824545,1.369289
STERLING ST,19611601,11082718,-8528883,0.565110
BEVERLY RD,85431731,18828090,-66603641,0.220388
GUN HILL RD,23539559,10125233,-13414326,0.430137


In [ ]:
mta'